## リスト8-1 最長増加部分列問題に対する単純な解

In [1]:
from itertools import combinations

def naive_lis(seq):
    for length in range(len(seq), 0, -1):       # n, n-1, ... , 1
        for sub in combinations(seq, length):   # 与えられた列の部分列
            if list(sub) == sorted(sub):        # 増加している列かどうかを確認
                return sub                      # そうであれば、それを返す

In [2]:
seq = [3, 1, 0, 2, 4]
naive_lis(seq)
# (1, 2, 4)

(1, 2, 4)

In [37]:
naive_lis([1, 0, 7, 2, 8, 3, 4, 9, 5, 6])
# (1, 2, 3, 4, 5, 6)

(1, 2, 3, 4, 5, 6)

# 8-1 DRY(Don't Repeat Yourself)の原則：繰り返しを避ける

## リスト8-2 メモ化デコレータ

In [5]:
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

In [3]:
from functools import wraps

def memo(func):
    cache = {}                                  # 部分問題の解をキャッシュする
    @wraps(func)                                # wrapをfuncのように見せる
    def wrap(*args):                            # メモ化するラッパ関数
        if args not in cache:                   # すでに計算されているか?
            cache[args] = func(*args)           # されていない場合、計算し、その解をキャッシュ
        return cache[args]                      # キャッシュされている解を返す
    return wrap                                 # ラッパ関数を返す

In [6]:
fib = memo(fib)
print(fib(100))
# 573147844013817084101

573147844013817084101


In [7]:
@memo
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)
fib(100)
# 573147844013817084101

573147844013817084101

In [8]:
@memo
def two_pow(i):
    if i == 0: return 1
    return two_pow(i-1) + two_pow(i-1)

In [9]:
two_pow(10)
# 1024

1024

In [10]:
print(two_pow(100))
# 1267650600228229401496703205376

1267650600228229401496703205376


In [11]:
def two_pow(i):
    if i == 0: return 1
    return 2*two_pow(i-1)
two_pow(10)
# 1024

1024

In [13]:
print(two_pow(100))
# 1267650600228229401496703205376

1267650600228229401496703205376


In [14]:
@memo
def C(n,k):
    if k == 0: return 1
    if n == 0: return 0
    return C(n-1,k-1) + C(n-1,k)
C(4,2)
# 6

6

In [15]:
print(C(100,50))
# 100891344545564193334812497256

100891344545564193334812497256


In [16]:
C(10,7)
# 120

120

In [18]:
C(4, 4)
# 1

1

In [19]:
C(4, 5)
# 0

0

In [20]:
from collections import defaultdict
n, k = 10, 7
C = defaultdict(int)
for row in range(n+1):
    C[row,0] = 1
    for col in range(1,k+1):
        C[row,col] = C[row-1,col-1] + C[row-1,col]

C[n,k]
# 120

120

## 8-2 有向非巡回グラフにおける最短経路

## リスト8-3 メモ化を使った再帰的DAG最短経路

In [21]:
def rec_dag_sp(W, s, t):                        # sからtへの最短経路
    @memo                                       # dをメモ化
    def d(u):                                   # uからtまでの距離
        if u == t: return 0                     # 到着している場合
        return min(W[u][v]+d(v) for v in W[u])  # 最初のステップの中でベストの選択肢
    return d(s)                                 # dを実際の開始ノードsに適用

In [24]:
DAG = {
    'a': {'b':0},
    'b': {'c':4, 'd':6},
    'c': {'g':2, 'h':-6},
    'd': {'f':3, 'e':5},
    'e': {'g':0, 'h':-6},
    'f': {'i':-1},
    'g': {'h':4},
    'h': {'i':7},
    'i': {}
}

In [25]:
rec_dag_sp(DAG, 'a', 'i')
# 5

5

## リスト8-4 DAG最短経路

In [27]:
def dag_sp(W, s, t):                            # sからtまでの最短経路
    d = {u:float('inf') for u in W}             # 距離の推定
    d[s] = 0                                    # 開始ノード: 距離は0
    for u in topsort(W):                        # トポロジカルソートの順序で
        if u == t: break                        # すでに訪問済みの場合
        for v in W[u]:                          # 各出力エッジに対して
            d[v] = min(d[v], d[u] + W[u][v])    # エッジを緩和
    return d[t]    

In [29]:
# 第4章より
def topsort(G):
    count = dict((u, 0) for u in G)             # 各ノードに対する入力次数
    for u in G:
        for v in G[u]:
            count[v] += 1                       # すべての入力エッジを数える
    Q = [u for u in G if count[u] == 0]         # 最初の有向なノード
    S = []                                      # 結果
    while Q:                                    # スタートノードがある限りwhileを継続
        u = Q.pop()                             # 1つを抽出
        S.append(u)                             # 抽出したものを残りの最初として使う
        for v in G[u]:
            count[v] -= 1                       # 出力エッジ分"数え上げ"を取り消す
            if count[v] == 0:                   # 新しい有向なスタートノードがある場合
                Q.append(v)                     # 次に扱う候補に追加
    return S

In [30]:
dag_sp(DAG, 'a', 'i')
# 5

5

# 8−3 最長増加部分列

## リスト8-5 最長増加部分列問題に対するメモ化と再帰を使った解

In [31]:
def rec_lis(seq):                               # 最長増加部分列
    @memo
    def L(cur):                                 # seq[cur]で終わる最長部分列
        res = 1                                 # 長さが少なくとも1
        for pre in range(cur):                  # 先行ノードの候補
            if seq[pre] <= seq[cur]:            # 有効な（小さい）先行ノードの場合
                res = max(res, 1 + L(pre))      # 解を改善するでしょうか？
        return res
    return max(L(i) for i in range(len(seq)))   # すべての中で一番長いもの

In [32]:
seq = [3, 1, 0, 2, 4]
rec_lis(seq)
# 3

3

## リスト8-6 最長増加部分列問題に対する反復を使った基本的な解

In [33]:
def basic_lis(seq):
    L = [1] * len(seq)
    for cur, val in enumerate(seq):
        for pre in range(cur):
            if seq[pre] <= val:
                L[cur] = max(L[cur], 1 + L[pre])
    return max(L)

In [34]:
basic_lis(seq)
# 3

3

## リスト8-7 最長増加部分列

In [35]:
from bisect import bisect

def lis(seq):                                   # 最長増加部分列
    end = []                                    # すべての長さに対する最後尾の値
    for val in seq:                             # すべての値を順に試す
        idx = bisect(end, val)                  # 最後尾上に構築できるか
        if idx == len(end): end.append(val)     # 最長部分列を拡張
        else: end[idx] = val                    # 前の最後尾の値を減らす
    return len(end)                             # 最長部分列を発見

In [36]:
lis(seq)
# 3

3

In [38]:
from random import *
seqs = [[randrange(100) for i in range(5+i)] for i in range(10)]
seqs.append([1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 4, 3, 3, 3, 4, 4, 4])
for seq in seqs:
    res = naive_lis(seq)
    for f in [basic_lis, rec_lis, lis]:
        res2 = f(seq)
        assert res2 == len(res), (res, seq, res2, f)

# 8-4 列の比較

## リスト8-8 メモ化を使ったLCS問題に対する再帰的な解

In [39]:
def rec_lcs(a,b):                               # 最長共通部分列
    @memo                                       # Lをメモ化
    def L(i,j):                                 # プレフィックスはa[:i] と b[:j]
        if min(i,j) < 0: return 0               # どちらかのプレフィックスが空の場合
        if a[i] == b[j]: return 1 + L(i-1,j-1)  # 等しい場合、対角線上に移動
        return max(L(i-1,j), L(i,j-1))          # a[i]かb[j]を取り除く
    return L(len(a)-1,len(b)-1)                 # 全体の列に対してLを実行

In [41]:
rec_lcs("spock", "asoka")
# 3

3

In [42]:
rec_lcs("AGCGA", "CAGATAGAG")
# 4

4

In [43]:
rec_lcs("Starbuck", "Starwalker")
# 5

5

## リスト8-9 最長共通部分列（LCS）問題に対する反復を使った解

In [44]:
def lcs(a,b):
    n, m = len(a), len(b)
    pre, cur = [0]*(n+1), [0]*(n+1)             # 前と現在の行
    for j in range(1,m+1):                      # b上を走査
        pre, cur = cur, pre                     # 前の行を保存, 現在の値を上書き
        for i in range(1,n+1):                  # a上を走査
            if a[i-1] == b[j-1]:                # プレフィックスの最後の要素が等しい場合
                cur[i] = pre[i-1] + 1           # L(i,j) = L(i-1,j-1) + 1
            else:                               # そうでない場合、
                cur[i] = max(pre[i], cur[i-1])  # max(L(i,j-1),L(i-1,j))
    return cur[n]                               # L(n,m)と等しい

In [45]:
lcs("spock", "asoka")
# 3

3

In [46]:
lcs("AGCGA", "CAGATAGAG")
# 4

4

In [47]:
lcs("Starbuck", "Starwalker")
# 5

5

# 8-5 ナップサック問題の反撃

## リスト8-10 制限なし整数ナップサック問題に対するメモ化を使った再帰的な解

In [49]:
def rec_unbounded_knapsack(w, v, c):            # 重みw、価値v、容量c
    @memo                                       # mはメモ化されている
    def m(r):                                   # 残りの容量に対して最大の値を返す
        if r == 0: return 0                     # 容量がない場合、価値は0
        val = m(r-1)                            # 最後の単位容量を無視
        for i, wi in enumerate(w):              # すべてのオブジェクトを試す
            if wi > r: continue                 # 重すぎる場合は無視
            val = max(val, v[i] + m(r-wi))      # 価値を足し、重みを引く
        return val                              # 最後のオブジェクトの中で最大値を返す
    return m(c)                                 # 全容量が可能な場合で計算

## リスト8-11 制限なし整数ナップサック問題に対する反復を使った解

In [50]:
def unbounded_knapsack(w, v, c):
    m = [0]
    for r in range(1,c+1):
        val = m[r-1]
        for i, wi in enumerate(w):
            if wi > r: continue
            val = max(val, v[i] + m[r-wi])
        m.append(val)
    return m[c]

In [51]:
funcs = [rec_unbounded_knapsack, unbounded_knapsack]
w, v = [1, 2], [2, 5]
[f(w, v, 5) for f in funcs]
# [12, 12]

[12, 12]

In [52]:
w, v = [3, 2, 4], [5, 4, 2]
[f(w, v, 7) for f in funcs]
# [13, 13]

[13, 13]

## リスト8-12 0−1 ナップサック問題に対するメモ化を使った再帰的な解

In [53]:
def rec_knapsack(w, v, c):                      # 重みw, 価値v, 容量c
    @memo                                       # メモ化されているm
    def m(k, r):                                # k個のオブジェクトと容量rに対する最大値v
        if k == 0 or r == 0: return 0           # オブジェクト0, 容量0
        i = k-1                                 # 検討中のオブジェクト
        drop = m(k-1, r)                        # オブジェクトを除いたとしたら
        if w[i] > r: return drop                # 重すぎる場合、除く
        return max(drop, v[i] + m(k-1, r-w[i])) # 入れた場合と入れなかった場合で大きい方
    return m(len(w), c)                         # すべてのオブジェクト、すべての容量に対して

## リスト8-13 0−1 ナップサック問題に対する反復を使った解

Note: 原文のgithubには`knapsack_wrap`, `knapsack_old`, `brutish_knapsack`なども実装されている。

In [54]:
def knapsack(w, v, c):                          # 解を表す行列を返す
    n = len(w)                                  # 使用可能なアイテムの数
    m = [[0]*(c+1) for i in range(n+1)]         # 最大値を格納する空の行列
    P = [[False]*(c+1) for i in range(n+1)]     # 取捨選択を格納する行列
    for k in range(1,n+1):                      # 最初のkオブジェクトを使える
        i = k-1                                 # 検討中のオブジェクト
        for r in range(1,c+1):                  # すべて正の容量
            m[k][r] = drop = m[k-1][r]          # デフォルトは除去する
            if w[i] > r: continue               # 重すぎる場合、無視
            keep = v[i] + m[k-1][r-w[i]]        # 保持する値
            m[k][r] = max(drop, keep)           # 保持と除去の結果が良い方
            P[k][r] = keep > drop               # 保持をしたかどうか
    return m, P                                 # 結果すべてを返す

In [60]:
w, v, c = [2, 3, 4, 5], [3, 4, 5, 6], 5
m, P = knapsack(w, v, c)
k, r, items = len(w), c, set()
while k > 0 and r > 0:
    i = k-1
    if P[k][r]:
        items.add(i)
        r -= w[i]
    k -= 1

sorted(items)
# [0, 1]

[0, 1]

# 8-6 二値列分割

## リスト8-14 想定最適探索コストに対するメモ化を使った再帰関数

In [55]:
def rec_opt_tree(p):
    @memo
    def s(i,j):
        if i == j: return 0
        return s(i,j-1) + p[j-1]
    @memo
    def e(i,j):
        if i == j: return 0
        sub = min(e(i,r) + e(r+1,j) for r in range(i,j))
        return sub + s(i,j)
    return e(0,len(p))

In [57]:
w = [0.25, 0.2, 0.05, 0.2, 0.3]
rec_opt_tree(w)
# 2.1

2.1

In [56]:
from collections import defaultdict

def opt_tree(p):
    n = len(p)
    s, e = defaultdict(int), defaultdict(int)
    for k in range(1,n+1):
        for i in range(n-k+1):
            j = i + k
            s[i,j] = s[i,j-1] + p[j-1]
            e[i,j] = min(e[i,r] + e[r+1,j] for r in range(i,j))
            e[i,j] += s[i,j]
    return e[0,n]

In [58]:
opt_tree(w)
# 2.1

2.1

In [59]:
from random import *
ws = [[random() for i in range(randrange(4,9))] for j in range(20)]
for w in ws:
    assert rec_opt_tree(w) == opt_tree(w)